In [20]:
import torch
from models import skipgram
import pandas as pd
import torch.nn.functional as F

In [22]:
vocab = pd.read_csv('./data/vocab.generated.csv', index_col='token')

def getIdFromToken(token: str):
    try:
        return int(vocab.at[token, 'id'])
    except:
        return int(vocab.at['unk', 'id'])

def getTokenFromId(id: float):
    return vocab[vocab['id'] == id].index

In [18]:

model = skipgram.Model(vocab.size + 1, skipgram.EMBEDDING_DIM)

state_dict = torch.load('./data/weights.generated.pt')
# model.load_state_dict(weights)
# model.eval()

print(state_dict)


OrderedDict([('embeddings.weight', tensor([[ 1.9269,  1.4873,  0.9007,  ..., -0.4879, -0.9138, -0.6581],
        [ 0.0780,  0.5258, -0.4880,  ...,  0.4880,  0.7846,  0.0286],
        [ 0.6408,  0.5832,  1.0669,  ...,  1.4506,  0.2695, -0.2104],
        ...,
        [-0.9871,  0.1414, -0.0884,  ...,  0.1128, -0.6210, -0.6534],
        [-0.2218, -1.2008,  1.3206,  ...,  0.1845,  0.2944, -1.4129],
        [ 0.4092,  1.3424,  1.0782,  ..., -1.3501, -0.6740, -0.4540]])), ('linear.weight', tensor([[-0.3266, -0.2425, -0.2588,  ...,  0.3584,  0.4915, -0.3154],
        [-0.4095, -0.2458, -0.1848,  ...,  0.2910,  0.3717, -0.2463],
        [-0.3900, -0.3171, -0.2505,  ...,  0.3838,  0.4324, -0.2738],
        ...,
        [-0.4033, -0.2710, -0.3405,  ...,  0.2478,  0.4471, -0.2722],
        [-0.3304, -0.1853, -0.2074,  ...,  0.2568,  0.5291, -0.2021],
        [ 0.3876,  0.3989,  0.3868,  ..., -0.5307, -0.2387, -0.2268]])), ('linear.bias', tensor([-0.4209, -0.4239, -0.5138,  ..., -0.4904, -0.4274, 

/var/folders/1y/mkp0dwnj2hb98l4zjyg2fjfc0000gn/T/ipykernel_87825/1703795876.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./data/weights.gener

In [23]:
#find the n most similar words
def most_similar(word, n=5):
    word_idx = getIdFromToken(word)
    A =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
    word_similarities = []
    for i in range(len(vocab)):
        B =  model.embeddings.weight[i].unsqueeze(0)
        cosine_similarity = F.cosine_similarity(A, B, dim=1)
        word_similarities.append((getTokenFromId(i), cosine_similarity.item()))
    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    return word_similarities[:n]

most_similar('python')

[(Index(['murderers'], dtype='object', name='token'), 0.5212107300758362),
 (Index(['wavefunction'], dtype='object', name='token'), 0.5169843435287476),
 (Index(['accenture'], dtype='object', name='token'), 0.5166984796524048),
 (Index(['icaza'], dtype='object', name='token'), 0.5114834308624268),
 (Index(['parallelized'], dtype='object', name='token'), 0.5112743973731995)]